In [1]:

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import math
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preparation import fetch_dataset
from src.processing import split_train_test_val, dataset_to_array, dataset_to_generator, make_tf_dataset
from src.modules.visualizer import do_heatmap
import numpy as np

In [2]:
dataset_a = fetch_dataset('dataset_a1_slim.pkl')
dataset_f = fetch_dataset('dataset_f1_slim.pkl')
dataset_g = fetch_dataset('dataset_g1_slim.pkl')
dataset_all = fetch_dataset('dataset_all_slim.pkl')

train, test, val = split_train_test_val(dataset_all)


# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
batch_size = 25


In [3]:
x_train, y_train = dataset_to_array(train)
x_test, y_test = dataset_to_array(test)
x_val, y_val = dataset_to_array(val)

In [4]:

data_train = make_tf_dataset(train, window_size, num_of_features, label_length).batch(batch_size)
data_test = make_tf_dataset(test, window_size, num_of_features, label_length).batch(batch_size)
data_val = make_tf_dataset(val, window_size, num_of_features, label_length).batch(batch_size)


W0729 00:49:37.386618 4660827584 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [5]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
# Clear any logs from previous runs
!rm - rf ./logs/

rm: -: No such file or directory
rm: rf: No such file or directory
rm: ./logs/: is a directory


In [6]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([128, 256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [7]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM
def train_test_model(hparams):
    # Create a model
    model = Sequential()


    model.add(LSTM(hparams[HP_NUM_UNITS], return_sequences=False, input_shape=(window_size, num_of_features)))
    model.add(Dropout(hparams[HP_DROPOUT]))
    model.add(Flatten())
    model.add(Dense(label_length, activation='softmax'))
    model.compile(loss='binary_crossentlropy',
                        optimizer=hparams[HP_OPTIMIZER],
                       metrics=['accuracy',])
    
    
 
    
    # Run with 1 epoch to speed things up for demo purposes
    model.fit(data_train, epochs=40)
    _, accuracy = model.evaluate(data_test)
    return accuracy


def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [8]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}


W0729 00:49:38.241136 4660827584 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
2386/2386 [==============================] - 49s 21ms/step - loss: 0.2998 - accuracy: 0.8863
Epoch 2/40
2386/2386 [==============================] - 40s 17ms/step - loss: 0.2107 - accuracy: 0.9227
Epoch 3/40
2386/2386 [==============================] - 41s 17ms/step - loss: 0.1877 - accuracy: 0.9306
Epoch 4/40
2386/2386 [==============================] - 41s 17ms/step - loss: 0.1698 - accuracy: 0.9374
Epoch 5/40
2386/2386 [==============================] - 40s 17ms/step - loss: 0.1557 - accuracy: 0.9425
Epoch 6/40
2386/2386 [==============================] - 41s 17ms/step - loss: 0.1486 - accuracy: 0.9457
Epoch 7/40
2386/2386 [==============================] - 43s 18ms/step - loss: 0.1385 - accuracy: 0.9491
Epoch 8/40
2386/2386 [==============================] - 45s 19ms/step - loss: 0.1317 - accuracy: 0.9510
Epoch 9/40
2386/2386 [==============================] - 44s 18ms/step - loss: 0.1227 - accuracy: 0.9546
Epoch 10/40
2386/2386 [==============================] - 48s 20m

W0729 02:21:30.061506 4660827584 nn_ops.py:4220] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Epoch 1/40
2386/2386 [==============================] - 55s 23ms/step - loss: 0.3693 - accuracy: 0.8526
Epoch 2/40
2386/2386 [==============================] - 44s 19ms/step - loss: 0.2798 - accuracy: 0.9000
Epoch 3/40
2386/2386 [==============================] - 44s 18ms/step - loss: 0.2508 - accuracy: 0.9142
Epoch 4/40
2386/2386 [==============================] - 44s 19ms/step - loss: 0.2359 - accuracy: 0.9195
Epoch 5/40
2386/2386 [==============================] - 44s 18ms/step - loss: 0.2229 - accuracy: 0.9240
Epoch 6/40
2386/2386 [==============================] - 43s 18ms/step - loss: 0.2145 - accuracy: 0.9263
Epoch 7/40
2386/2386 [==============================] - 42s 18ms/step - loss: 0.2132 - accuracy: 0.9257
Epoch 8/40
2386/2386 [==============================] - 42s 18ms/step - loss: 0.2057 - accuracy: 0.9282
Epoch 9/40
2386/2386 [==============================] - 41s 17ms/step - loss: 0.1929 - accuracy: 0.9330
Epoch 10/40
2386/2386 [==============================] - 41s 17m

...

Epoch 1/40
2386/2386 [==============================] - 71s 30ms/step - loss: 0.2134 - accuracy: 0.9281
Epoch 2/40
2386/2386 [==============================] - 57s 24ms/step - loss: 0.1567 - accuracy: 0.9476
Epoch 3/40
2386/2386 [==============================] - 57s 24ms/step - loss: 0.1433 - accuracy: 0.9522
Epoch 4/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1353 - accuracy: 0.9543
Epoch 5/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1295 - accuracy: 0.9568
Epoch 6/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1248 - accuracy: 0.9586
Epoch 7/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1198 - accuracy: 0.9613
Epoch 8/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1182 - accuracy: 0.9622
Epoch 9/40
2386/2386 [==============================] - 58s 24ms/step - loss: 0.1192 - accuracy: 0.9616
Epoch 10/40
2386/2386 [==============================] - 58s 24m

2386/2386 [==============================] - 110s 46ms/step - loss: 0.0703 - accuracy: 0.9795
Epoch 32/40
2386/2386 [==============================] - 111s 46ms/step - loss: 0.0674 - accuracy: 0.9799
Epoch 33/40
2386/2386 [==============================] - 111s 47ms/step - loss: 0.0658 - accuracy: 0.9800
Epoch 34/40
2386/2386 [==============================] - 112s 47ms/step - loss: 0.0670 - accuracy: 0.9802
Epoch 35/40
2386/2386 [==============================] - 116s 48ms/step - loss: 0.0666 - accuracy: 0.9801
Epoch 36/40
2386/2386 [==============================] - 116s 49ms/step - loss: 0.0678 - accuracy: 0.9797
Epoch 37/40
2386/2386 [==============================] - 116s 49ms/step - loss: 0.0654 - accuracy: 0.9806
Epoch 38/40
2386/2386 [==============================] - 116s 49ms/step - loss: 0.0656 - accuracy: 0.9803
Epoch 39/40
2386/2386 [==============================] - 117s 49ms/step - loss: 0.0637 - accuracy: 0.9814
Epoch 40/40
2386/2386 [==============================] - 1

2386/2386 [==============================] - 125s 52ms/step - loss: 0.0762 - accuracy: 0.9735
Epoch 20/40
2386/2386 [==============================] - 108s 45ms/step - loss: 0.0753 - accuracy: 0.9738
Epoch 21/40
2386/2386 [==============================] - 109s 46ms/step - loss: 0.0727 - accuracy: 0.9751
Epoch 22/40
2386/2386 [==============================] - 118s 49ms/step - loss: 0.0704 - accuracy: 0.9754
Epoch 23/40
2386/2386 [==============================] - 129s 54ms/step - loss: 0.0678 - accuracy: 0.9766
Epoch 24/40
2386/2386 [==============================] - 117s 49ms/step - loss: 0.0656 - accuracy: 0.9774
Epoch 25/40
2386/2386 [==============================] - 119s 50ms/step - loss: 0.0641 - accuracy: 0.9783
Epoch 26/40
2386/2386 [==============================] - 116s 49ms/step - loss: 0.0642 - accuracy: 0.9778
Epoch 27/40
2386/2386 [==============================] - 125s 52ms/step - loss: 0.0619 - accuracy: 0.9787
Epoch 28/40
2386/2386 [==============================] - 1

2386/2386 [==============================] - 702s 294ms/step - loss: 0.1038 - accuracy: 0.9610
Epoch 8/40
2386/2386 [==============================] - 710s 297ms/step - loss: 0.0974 - accuracy: 0.9640
Epoch 9/40
2386/2386 [==============================] - 708s 297ms/step - loss: 0.0926 - accuracy: 0.9664
Epoch 10/40
2386/2386 [==============================] - 711s 298ms/step - loss: 0.0888 - accuracy: 0.9678
Epoch 11/40
2386/2386 [==============================] - 699s 293ms/step - loss: 0.0807 - accuracy: 0.9702
Epoch 12/40
2386/2386 [==============================] - 767s 322ms/step - loss: 0.0800 - accuracy: 0.9706
Epoch 13/40
2386/2386 [==============================] - 1051s 441ms/step - loss: 0.0705 - accuracy: 0.9745
Epoch 14/40
2386/2386 [==============================] - 802s 336ms/step - loss: 0.0697 - accuracy: 0.9748
Epoch 15/40
2386/2386 [==============================] - 808s 339ms/step - loss: 0.0647 - accuracy: 0.9767
Epoch 16/40
2386/2386 [===========================

2386/2386 [==============================] - 741s 310ms/step - loss: 0.2496 - accuracy: 0.9067
Epoch 2/40
2386/2386 [==============================] - 473s 198ms/step - loss: 0.1812 - accuracy: 0.9342
Epoch 3/40
2386/2386 [==============================] - 391s 164ms/step - loss: 0.1518 - accuracy: 0.9451
Epoch 4/40
2386/2386 [==============================] - 389s 163ms/step - loss: 0.1369 - accuracy: 0.9506
Epoch 5/40
2386/2386 [==============================] - 392s 164ms/step - loss: 0.1251 - accuracy: 0.9551
Epoch 6/40
2386/2386 [==============================] - 391s 164ms/step - loss: 0.1163 - accuracy: 0.9583
Epoch 7/40
2386/2386 [==============================] - 391s 164ms/step - loss: 0.1087 - accuracy: 0.9611
Epoch 8/40
2386/2386 [==============================] - 389s 163ms/step - loss: 0.1026 - accuracy: 0.9628
Epoch 9/40
2386/2386 [==============================] - 392s 164ms/step - loss: 0.0984 - accuracy: 0.9650
Epoch 10/40
2386/2386 [==============================] - 

In [13]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 18620), started 0:01:36 ago. (Use '!kill 18620' to kill it.)

###  - When are you going to get back to work?
###  - When did you think I stopped?
